# Testing postprocessing / plots for nonresonant analysis

In [ ]:
import utils
import plotting
import postprocessing
import corrections
from collections import OrderedDict

from utils import CUT_MAX_VAL, ShapeVar
from HHbbVV.hh_vars import (
    years,
    data_key,
    qcd_key,
    bg_keys,
    samples,
    nonres_sig_keys,
    nonres_samples,
    txbb_wps,
    jec_shifts,
    jmsr_shifts,
    LUMI,
)
from postprocessing import nonres_shape_vars

import numpy as np
import pandas as pd
import pickle
from pandas.errors import SettingWithCopyWarning
import hist
from hist import Hist

import os
from copy import deepcopy
from inspect import cleandoc
import warnings

import matplotlib.pyplot as plt
import mplhep as hep
import matplotlib.ticker as mticker

plt.style.use(hep.style.CMS)
hep.style.use("CMS")
formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))
plt.rcParams.update({"font.size": 16})

# ignore these because they don't seem to apply
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

MAIN_DIR = Path("../../../")
samples_dir = MAIN_DIR / "../data/skimmer/24Mar14UpdateData"
year = "2018"
bdt_preds_dir = samples_dir / "24_04_05_k2v0_training_eqsig_vbf_vars_rm_deta/inferences"

date = "24May16"
plot_dir = MAIN_DIR / f"plots/PostProcessing/{date}/"
templates_dir = f"templates/{date}"
_ = os.system(f"mkdir -p {plot_dir}")
_ = os.system(f"mkdir -p {plot_dir}/cutflows/")
_ = os.system(f"mkdir -p {plot_dir}/ControlPlots/{year}/")
# _ = os.system(f"mkdir -p {plot_dir}/templates/")
# _ = os.system(f"mkdir -p {plot_dir}/templates/wshifts")
# _ = os.system(f"mkdir -p {plot_dir}/templates/jshifts")
# _ = os.system(f"mkdir -p {templates_dir}")

selection_regions = postprocessing.get_nonres_selection_regions(year)

In [ ]:
# add both VBF keys just in case (the unused one will be removed below)
nonres_samples = {
    "HHbbVV": "GluGluToHHTobbVV_node_cHHH1",
    "qqHH_CV_1_C2V_0_kl_1_HHbbVV": "VBF_HHTobbVV_CV_1_C2V_0_C3_1",
    # "qqHH_CV_1_C2V_2_kl_1_HHbbVV": "VBF_HHTobbVV_CV_1_C2V_2_C3_1",
}

# bg_keys = ["QCD", "TT", "Data"]
# samples = {key: val for key, val in samples.items() if key in bg_keys}

## Load samples

In [ ]:
filters = postprocessing.load_filters
systematics = {year: {}}

# save cutflow as pandas table
cutflow = pd.DataFrame(index=list(samples.keys()) + list(nonres_samples.keys()))

events_dict = postprocessing.load_samples(
    samples_dir, {**nonres_samples, **samples}, year, filters, hem_cleaning=False, variations=False
)
# events_dict |= postprocessing.load_samples(samples_dir, {"Data": "JetHT"}, year, filters, hem_cleaning=False)

utils.add_to_cutflow(events_dict, "Preselection", "finalWeight", cutflow)
cutflow

### Scale factors and bb VV assignment

In [ ]:
bb_masks = postprocessing.bb_VV_assignment(events_dict)
postprocessing.derive_variables(events_dict, bb_masks, nonres_vars=True, do_jshifts=False)
postprocessing.qcd_sf(events_dict, cutflow)
# events_dict[sig_key] = postprocessing.postprocess_lpsfs(events_dict[sig_key])
cutflow

In [ ]:
postprocessing.load_bdt_preds(events_dict, year, bdt_preds_dir, jec_jmsr_shifts=True)

## Control plots

In [ ]:
sel, _ = utils.make_selection(
    {
        "bbFatJetPt": [450, CUT_MAX_VAL],
        "VVFatJetPt": [450, CUT_MAX_VAL],
        "vbf_Mass_jj": [500, CUT_MAX_VAL],
    },
    events_dict,
    bb_masks,
)

In [ ]:
control_plot_vars = [
    # ShapeVar(var="MET_pt", label=r"$p^{miss}_T$ (GeV)", bins=[50, 0, 300]),
    # ShapeVar(var="DijetEta", label=r"$\eta^{jj}$", bins=[30, -8, 8]),
    # ShapeVar(var="DijetPt", label=r"$p_T^{jj}$ (GeV)", bins=[30, 0, 750]),
    # ShapeVar(var="DijetMass", label=r"$m^{jj}$ (GeV)", bins=[30, 600, 4000]),
    # ShapeVar(var="bbFatJetEta", label=r"$\eta^{bb}$", bins=[20, -2.4, 2.4]),
    # ShapeVar(
    #     var="bbFatJetPt", label=r"$p^{bb}_T$ (GeV)", bins=[20, 300, 2300], significance_dir="right"
    # ),
    # ShapeVar(
    #     var="bbFatJetParticleNetMass",
    #     label=r"$m^{bb}_{reg}$ (GeV)",
    #     bins=[20, 50, 250],
    #     significance_dir="bin",
    # ),
    # ShapeVar(var="bbFatJetMsd", label=r"$m^{bb}_{msd}$ (GeV)", bins=[50, 0, 300]),
    # ShapeVar(var="bbFatJetParticleNetMD_Txbb", label=r"$T^{bb}_{Xbb}$", bins=[50, 0.8, 1]),
    # ShapeVar(var="VVFatJetEta", label=r"$\eta^{VV}$", bins=[20, -2.4, 2.4]),
    # ShapeVar(var="VVFatJetPt", label=r"$p^{VV}_T$ (GeV)", bins=[20, 300, 2300]),
    # ShapeVar(var="VVFatJetParticleNetMass", label=r"$m^{VV}_{reg}$ (GeV)", bins=[20, 50, 250]),
    # ShapeVar(var="VVFatJetMsd", label=r"$m^{VV}_{msd}$ (GeV)", bins=[40, 50, 250]),
    # ShapeVar(var="VVFatJetParticleNet_Th4q", label=r"Prob($H \to 4q$) vs Prob(QCD) (Non-MD)", bins=[50, 0, 1]),
    # ShapeVar(
    #     var="VVFatJetParTMD_THWW4q",
    #     label=r"Prob($H \to VV \to 4q$) vs Prob(QCD) (Mass-Decorrelated)",
    #     bins=[50, 0, 1],
    # ),
    # ShapeVar(var="VVFatJetParTMD_probT", label=r"Prob(Top) (Mass-Decorrelated)", bins=[50, 0, 1]),
    # ShapeVar(var="VVFatJetParTMD_THWWvsT", label=r"$T^{VV}_{HWW}$", bins=[50, 0, 1]),
    # ShapeVar(var="bbFatJetPtOverDijetPt", label=r"$p^{bb}_T / p_T^{jj}$", bins=[50, 0, 40]),
    # ShapeVar(var="VVFatJetPtOverDijetPt", label=r"$p^{VV}_T / p_T^{jj}$", bins=[50, 0, 40]),
    # ShapeVar(var="VVFatJetPtOverbbFatJetPt", label=r"$p^{VV}_T / p^{bb}_T$", bins=[50, 0.4, 2.0]),
    # ShapeVar(var="nGoodMuons", label=r"# of Muons", bins=[3, 0, 3]),
    # ShapeVar(var="nGoodElectrons", label=r"# of Electrons", bins=[3, 0, 3]),
    # ShapeVar(var="nGoodJets", label=r"# of AK4 B-Jets", bins=[5, 0, 5]),
    ShapeVar(var="VBFJetPt0", label=r"Leading VBF-tagged Jet $p_T$", bins=[20, 20, 300]),
    ShapeVar(var="VBFJetPt1", label=r"Sub-leading VBF-tagged Jet $p_T$", bins=[20, 20, 300]),
    ShapeVar(var="VBFJetEta0", label=r"Leading VBF-tagged Jet $\eta$", bins=[9, -4.5, 4.5]),
    ShapeVar(var="VBFJetEta1", label=r"Sub-leading VBF-tagged Jet $\eta$", bins=[9, -4.5, 4.5]),
    ShapeVar(var="VBFJetPhi0", label=r"Leading VBF-tagged Jet $\varphi$", bins=[10, -3, 3]),
    ShapeVar(var="VBFJetPhi1", label=r"Sub-leading VBF-tagged Jet $\varphi$", bins=[10, -3, 3]),
    ShapeVar(var="vbf_Mass_jj", label=r"$m_{jj}^{VBF}$", bins=[20, 0, 1000]),
    ShapeVar(var="vbf_dEta_jj", label=r"$|\Delta\eta_{jj}^{VBF}|$", bins=[20, 0, 6]),
    # ShapeVar(var="BDTScore", label=r"BDT Score", bins=[50, 0, 1]),
]

hists = postprocessing.control_plots(
    events_dict,
    bb_masks,
    # ["HHbbVV", "qqHH_CV_1_C2V_1_kl_1_HHbbVV"],
    ["HHbbVV"],
    control_plot_vars,
    plot_dir / f"ControlPlots/{year}/",
    year,
    # selection=sel,
    bg_keys=bg_keys,
    # bg_keys=["QCD", "TT", "ST", "V+Jets", "Hbb"],
    sig_scale_dict={"HHbbVV": 2e5, "qqHH_CV_1_C2V_1_kl_1_HHbbVV": 2e6},
    show=True,
    log=True,
)

In [ ]:
sel, _ = utils.make_selection(
    {"VVFatJetParTMD_THWWvsT": [0, 0.6], "bbFatJetParticleNetMD_Txbb": [0.999, CUT_MAX_VAL]},
    events_dict,
    bb_masks,
)


hists = postprocessing.control_plots(
    events_dict,
    bb_masks,
    ["HHbbVV", "qqHH_CV_1_C2V_1_kl_1_HHbbVV"],
    [
        ShapeVar(
            var="bbFatJetParticleNetMass",
            label=r"$m^{bb}_{reg}$ (GeV)",
            bins=[20, 50, 250],
            significance_dir="bin",
        )
    ],
    f"{plot_dir}/ControlPlots/{year}/",
    year,
    hists={},
    bg_keys=["QCD", "TT", "ST", "WJets", "ZJets", "Diboson"],
    selection=sel,
    sig_scale_dict={"HHbbVV": 1},
    combine_pdf=False,
    show=True,
)

Check mVV after BDT

In [ ]:
for bdt_cut in [0.0, 0.5, 0.9, 0.99, 0.995]:
    sel, _ = utils.make_selection(
        {"BDTScore": [bdt_cut, CUT_MAX_VAL]},
        events_dict,
        bb_masks,
    )

    hists = postprocessing.control_plots(
        events_dict,
        bb_masks,
        ["HHbbVV", "qqHH_CV_1_C2V_0_kl_1_HHbbVV"],
        [
            ShapeVar(
                var="VVFatJetParticleNetMass",
                label=r"$m^{VV}_{reg}$ (GeV)",
                bins=[20, 50, 250],
                significance_dir="bin",
            )
        ],
        plot_dir / f"ControlPlots/{year}/",
        year,
        cutstr=f"bdtcut_{bdt_cut}_",
        title=rf"$BDT_{{ggF}}$ ≥ {bdt_cut}" if bdt_cut != 0.0 else None,
        hists={},
        bg_keys=bg_keys,
        selection=sel,
        sig_scale_dict={"HHbbVV": 1},
        combine_pdf=False,
        show=True,
        log=False,
    )

Check BDT Sculpting

In [ ]:
postprocessing.plot_bdt_sculpting(events_dict, bb_masks, plot_dir, year, show=True)

In [ ]:
cuts = [0, 0.1, 0.5, 0.9, 0.95]
# bdtvars = ["", "TT", "VJets"]
bdtvars = [""]
plot_keys = ["TT"]

shape_var = ShapeVar(
    var="bbFatJetParticleNetMass", label=r"$m^{bb}_{reg}$ (GeV)", bins=[20, 50, 250]
)

for var in bdtvars:
    for key in plot_keys:
        ed_key = {key: events_dict[key]}
        bbm_key = {key: bb_masks[key]}

        fig, ax = plt.subplots(1, 1, figsize=(12, 12))
        plt.rcParams.update({"font.size": 24})

        for i, cut in enumerate(cuts):
            sel, _ = utils.make_selection({f"BDTScore{var}": [cut, CUT_MAX_VAL]}, ed_key, bbm_key)
            h = utils.singleVarHist(ed_key, shape_var, bbm_key, selection=sel)

            hep.histplot(
                h[key, ...] / np.sum(h[key, ...].values()),
                yerr=True,
                label=f"BDTScore >= {cut}",
                # density=True,
                ax=ax,
                linewidth=2,
                alpha=0.8,
            )

        ax.set_xlabel(shape_var.label)
        ax.set_ylabel("Fraction of Events")
        ax.legend()

        hep.cms.label(ax=ax, data=False, year=year, lumi=round(LUMI[year] / 1e3))
        plt.show()

In [ ]:
cuts = [0.01, 0.1, 0.5, 0.9, 0.99]
# bdtvars = ["", "TT", "VJets"]
bdtvars = [""]
sig_scales = [1e5, 5e4, 2e4, 1e4, 2e3]

# for ttcut in [0.01, 0.1, 0.5, 0.9, 0.99]:
#     ttsel, _ = utils.make_selection({"BDTScoreTT": [ttcut, CUT_MAX_VAL]}, events_dict, bb_masks)
#     cutstr = f"tt{ttcut}"

#     hists = postprocessing.control_plots(
#         events_dict,
#         bb_masks,
#         nonres_sig_keys,
#         control_plot_vars,
#         f"{plot_dir}/ControlPlots/{year}/",
#         year,
#         hists={},
#         bg_keys=["QCD", "TT", "ST", "V+Jets", "Diboson"],
#         selection=ttsel,
#         cutstr=cutstr,
#         show=True,
#     )

for var in bdtvars:
    for i, cut in enumerate(cuts):
        sel, _ = utils.make_selection({f"BDTScore{var}": [cut, CUT_MAX_VAL]}, events_dict, bb_masks)
        cutstr = f"bdt{var}{cut}"
        sig_scale = sig_scales[i]

        hists = postprocessing.control_plots(
            events_dict,
            bb_masks,
            nonres_sig_keys,
            control_plot_vars,
            f"{plot_dir}/ControlPlots/{year}/",
            year,
            hists={},
            bg_keys=["QCD", "TT", "ST", "V+Jets", "Diboson"],
            selection=sel,
            cutstr=cutstr,
            sig_scale_dict={"HHbbVV": sig_scale},
            combine_pdf=False,
            show=True,
        )

Overall BDT SF

In [ ]:
nonres_sig_keys

In [ ]:
postprocessing.lpsfs(
    events_dict,
    bb_masks,
    nonres_sig_keys,
    nonres_samples,
    cutflow,
    selection_regions["lpsf"],
    systematics,
    all_years=False,
)

In [ ]:
events = events_dict["HHbbVV"]
bb_mask = bb_masks["HHbbVV"]
weight = events["finalWeight"].values.squeeze()
weight_lp = weight * events["VV_lp_sf_nom"].values.squeeze()
weight_lp_sys_up = weight * events["VV_lp_sf_sys_up"].values.squeeze()
weight_lp_sys_down = weight * events["VV_lp_sf_sys_down"].values.squeeze()

plt.hist(
    utils.get_feat(events, "bbFatJetPt", bb_mask),
    np.linspace(250, 1200, 31),
    weights=weight,
    histtype="step",
    label="Pre-LP",
)
plt.hist(
    utils.get_feat(events, "bbFatJetPt", bb_mask),
    np.linspace(250, 1200, 31),
    weights=weight_lp,
    histtype="step",
    label="Post-LP",
)
plt.title("2018 HHbbVV")
plt.xlabel(r"$p_T^{VV}$ (GeV)")
plt.ylabel("Events")
# plt.hist(utils.get_feat(events, "VVFatJetPt", bb_mask), np.linspace(250, 2000, 31), weights=weight_lp_sys_up, histtype="step", label="Post-LP Sys Up")
# plt.hist(utils.get_feat(events, "VVFatJetPt", bb_mask), np.linspace(250, 2000, 31), weights=weight_lp_sys_down, histtype="step", label="Post-LP Sys Down")
plt.legend()

## Templates

In [ ]:
h, tsysts = postprocessing.get_templates(
    events_dict,
    bb_masks,
    year,
    nonres_sig_keys,
    selection_regions,
    nonres_shape_vars,
    systematics,
    bg_keys=bg_keys,
    plot_dir=f"{plot_dir}/templates",
    prev_cutflow=cutflow,
    weight_shifts={},
    jshift="",
    plot_shifts=False,
    show=True,
)

In [ ]:
templates = {}

for jshift in [""] + jec_shifts + jmsr_shifts:
    print(jshift)
    ttemps, tsyst = postprocessing.get_templates(
        events_dict,
        bb_masks,
        year,
        selection_regions[year],
        shape_var,
        shape_bins,
        blind_window,
        plot_dir=plot_dir,
        prev_cutflow=cutflow,
        weight_shifts=postprocessing.weight_shifts,
        jshift=jshift,
        show=False,
    )

    templates = {**templates, **ttemps}
    systematics = {**systematics, **tsyst}

In [ ]:
systematics

In [ ]:
templates_dict = {}

for year in years:
    with open(f"templates/Feb28/{year}_templates.pkl", "rb") as f:
        templates_dict[year] = pickle.load(f)

## Checking mass resolution and smearing

In [ ]:
def get_mass(events, shift="", i=0):
    if shift == "":
        mass = events["ak8FatJetParticleNetMass"][i]
    elif shift == "up":
        mass = events["ak8FatJetParticleNetMass_JMR_up"][i]
    elif shift == "down":
        mass = events["ak8FatJetParticleNetMass_JMR_down"][i]

    mass = mass[(mass > 100) * (mass < 150)]
    return mass

In [ ]:
events = events_dict["qqHH_CV_1_C2V_0_kl_1_HHbbVV"]
events = events_dict["qqHH_CV_1_C2V_2_kl_1_HHbbVV"]
events = events_dict["HHbbVV"]
for shift in ["", "up", "down"]:
    print(shift)
    mass = get_mass(events, shift)
    print("mean", np.mean(mass))
    print("std", np.std(mass))
    print("")

# print(np.std(events["ak8FatJetParticleNetMass_JMR_up"][0]) / np.std(events["ak8FatJetParticleNetMass"][0]))
# print(np.std(events["ak8FatJetParticleNetMass"][0]) / np.std(events["ak8FatJetParticleNetMass_JMR_down"][0]))

## Aside: checking if regressed mass is actually saving useful jets

In [ ]:
bbmsd = utils.get_feat(events_dict["HHbbVV"], "bbFatJetMsd", bb_masks["HHbbVV"])
vvmsd = utils.get_feat(events_dict["HHbbVV"], "VVFatJetMsd", bb_masks["HHbbVV"])
bbpnetm = utils.get_feat(events_dict["HHbbVV"], "bbFatJetParticleNetMass", bb_masks["HHbbVV"])
vvpnetm = utils.get_feat(events_dict["HHbbVV"], "VVFatJetParticleNetMass", bb_masks["HHbbVV"])
bbpnet = utils.get_feat(events_dict["HHbbVV"], "bbFatJetParticleNetMD_Xbb", bb_masks["HHbbVV"])
vvpart = utils.get_feat(events_dict["HHbbVV"], "VVFatJetParTMD_THWW4q", bb_masks["HHbbVV"])

In [ ]:
bbreg = (bbmsd < 50) * (bbpnetm > 50)
plt.title("Jets with Msd < 50 GeV and PNet Mass > 50 GeV")
plt.hist(bbpnet[bbreg], np.linspace(0, 1, 50), histtype="step", label="bb")
plt.xlabel("Txbb")
plt.show()

## Check signal jet matching with bb VV assignment

In [ ]:
events = events_dict["HHbbVV"]
bb_mask = bb_masks["HHbbVV"]
list(events.columns)

In [ ]:
print(np.mean(utils.get_feat(events, "bbFatJetHbb", bb_mask)))
print(np.mean(utils.get_feat(events, "bbFatJetHVV", bb_mask)))
print(np.mean(utils.get_feat(events, "VVFatJetHbb", bb_mask)))
print(np.mean(utils.get_feat(events, "VVFatJetHVV", bb_mask)))

In [ ]:
events = events_dict["HHbbVV"]
bb_mask = bb_masks["HHbbVV"]
weight_key = "finalWeight"

bbpnet_shape_var = ShapeVar(
    var="bbFatJetParticleNetMD_Txbb", label=r"$T^{bb}_{Xbb}$", bins=[20, 0.8, 1]
)
vvpnet_shape_var = ShapeVar(
    var="VVFatJetParticleNetMD_Txbb", label=r"$T^{VV}_{Xbb}$", bins=[20, 0, 1]
)

match_labels = [r"$bb$-matched", r"$VV$-matched", "Unmatched"]
colours = [plotting.colours["red"], plotting.colours["forestgreen"], plotting.colours["orange"]]

for jet, shape_var in [("bb", bbpnet_shape_var), ("VV", vvpnet_shape_var)]:
    h = Hist(
        hist.axis.StrCategory(match_labels, name="Sample"),
        shape_var.axis,
        storage="weight",
    )

    is_bb = utils.get_feat(events, f"{jet}FatJetHbb", bb_mask).astype(bool)
    is_VV = utils.get_feat(events, f"{jet}FatJetHVV", bb_mask).astype(bool)
    is_unmatched = ~(is_bb | is_VV)

    scores = utils.get_feat(events, shape_var.var, bb_mask)

    h.fill(match_labels[0], scores[is_bb], weight=events[weight_key][is_bb])
    h.fill(match_labels[1], scores[is_VV], weight=events[weight_key][is_VV])
    h.fill(match_labels[2], scores[is_unmatched], weight=events[weight_key][is_unmatched])

    fig, ax = plt.subplots(1, 1, figsize=(12, 12))
    plt.rcParams.update({"font.size": 24})
    hep.histplot(
        [h[label, :] for label in match_labels],
        color=colours,
        sort="yield",
        histtype="fill",
        stack=True,
        label=match_labels,
        ax=ax,
        yerr=True,
        # alpha=0.8,
        # hatch="//",
        edgecolor="black",
        linewidth=2,
    )
    ax.set_xlim(shape_var.bins[1], shape_var.bins[2])
    ax.legend(loc="upper left" if jet == "bb" else "upper right")
    ax.set_yscale("log")
    ax.set_xlabel(shape_var.label)
    ax.set_ylabel("Events")
    hep.cms.label(ax=ax, data=False, rlabel=None)
    plt.savefig(f"{plot_dir}/Xbb_{jet}_matching.pdf", bbox_inches="tight")
    plt.show()